In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
#Просто массив и его мини-анализ
array = np.random.randint(0,3, 20)
print(array)
# в numpy почему-то нет моды
print('mode:', stats.mode(array))
print('median:', np.median(array))
print('mean:', np.mean(array))

[2 1 0 1 1 0 2 0 1 0 0 1 1 0 0 0 0 2 0 0]
mode: ModeResult(mode=0, count=11)
median: 0.0
mean: 0.6


# Задания со второго урока с демо-курса по статистике

In [5]:
#Поиграем с датасетом с курса по статистике

taxi = pd.read_csv("Datasets/2_taxi_nyc.csv")

In [12]:
taxi.shape # Узнаем кол-во строк и столбцов

(29101, 14)

In [13]:
taxi.dtypes # Узнаем типы данных в сете

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [14]:
taxi.head(10)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
5,2015-01-01 01:00:00,Jan,Staten Island,6,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
6,2015-01-01 01:00:00,Jan,NaN,4,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
7,2015-01-01 02:00:00,Jan,Bronx,120,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
8,2015-01-01 02:00:00,Jan,Brooklyn,1229,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
9,2015-01-01 02:00:00,Jan,EWR,0,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0


In [17]:
# У нас есть неудобные колонки меняем их названия
taxi = taxi.rename(columns={'pcp 01':'pcp_01','pcp 06':'pcp_06','pcp 24':'pcp_24'})

In [16]:
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

In [19]:
# Узнаем сколько раз встречаются районы разными способами

taxi.query("borough == 'Brooklyn'").shape[0]

4343

In [20]:
taxi['borough'].value_counts()

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

In [22]:
#Узнаем общее число поездок
taxi['pickups'].sum()

14265773

In [25]:
taxi.groupby('borough').agg({'pickups':'sum'}) #Узнаем число поездок по районам

,pickups
borough,
Bronx,220047
Brooklyn,2321035
EWR,105
Manhattan,10367841
Queens,1343528
Staten Island,6957


In [29]:
#Ускоряем ответ - сделаем так, чтобы минимальные поездки нам сами вывелись
min_pickups = taxi.groupby('borough').agg({'pickups':'sum'}).idxmin()
print(min_pickups)

pickups    EWR
dtype: object


In [31]:
#Район города и выходной день
taxi.query("hday == 'Y'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,8027
Brooklyn,88011
EWR,7
Manhattan,340000
Queens,53562
Staten Island,250


In [37]:
taxi.query("hday == 'N'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,212020
Brooklyn,2233024
EWR,98
Manhattan,10027841
Queens,1289966
Staten Island,6707


In [38]:
taxi.query("hday == 'Y'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,8027
Brooklyn,88011
EWR,7
Manhattan,340000
Queens,53562
Staten Island,250


In [49]:
#Задание такое, надо определить в каких районах в выходные дни больше закозов
grouped_taxi = taxi.groupby(['borough', 'hday']).agg({'pickups': 'mean'}).reset_index()
grouped_taxi

,borough,hday,pickups
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
4,EWR,N,0.023467
5,EWR,Y,0.041916
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
8,Queens,N,308.899904
9,Queens,Y,320.730539


Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.

In [57]:
pickups_by_mon_bor = taxi.groupby(['pickup_month','borough']).agg({'pickups':'sum'}).sort_values('pickups',ascending = False).reset_index()

Теперь задача посложнее! Попрактикуемся в написании собственных функций и их применении к датафрейму. Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 

Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F и возвращает значения, переведённые в градусы Цельсия.

In [59]:
def temp_to_celcius(temp_series):
    return (temp_series - 32)*5/9

In [61]:
# Применяем функцию к колонке 'temp'
taxi['temp_celsius'] = temp_to_celcius(taxi['temp'])

# Выводим результат
print(taxi[['temp', 'temp_celsius']].head())

   temp  temp_celsius
0  30.0     -1.111111
1  30.0     -1.111111
2  30.0     -1.111111
3  30.0     -1.111111
4  30.0     -1.111111


# Минипроект со второго урока на курсе

In [2]:
bookings = pd.read_csv("Datasets/bookings.csv",sep = ';')

In [3]:
bookings.shape

(119390, 21)

In [4]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [5]:
bookings.head(10)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,FB,PRT,C,C,Transient,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015-07-01,2015,July,27,1,0,3,...,2,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-05-06
9,Resort Hotel,1,75,2015-07-01,2015,July,27,1,0,3,...,2,0.0,0,HB,PRT,D,D,Transient,Canceled,2015-04-22


In [6]:
#У нас в колонках пробелы, брр, надо это исправить
#taxi = taxi.rename(columns={'pcp 01':'pcp_01','pcp 06':'pcp_06','pcp 24':'pcp_24'})

# Мы могли бы так сделать, но это долго, ускорим процесс
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')
bookings.dtypes

hotel                         object
is_canceled                    int64
lead_time                      int64
arrival_full_date             object
arrival_date_year              int64
arrival_date_month            object
arrival_date_week_number       int64
arrival_date_day_of_month      int64
stays_in_weekend_nights        int64
stays_in_week_nights           int64
stays_total_nights             int64
adults                         int64
children                     float64
babies                         int64
meal                          object
country                       object
reserved_room_type            object
assigned_room_type            object
customer_type                 object
reservation_status            object
reservation_status_date       object
dtype: object

Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.

In [35]:
# Я спросил нейросеть про is_canceled = 0, она подсказала мне лямбда-функцию
bookings.groupby('country').agg({'is_canceled': lambda x: (x == 0).sum()}).reset_index().sort_values(by='is_canceled',ascending = False).head(5)

,country,is_canceled
135,PRT,21071
59,GBR,9676
56,FRA,8481
51,ESP,6391
43,DEU,6069


**Лидируют страны - ABW, AGO, AIA, ALB, AND**

На сколько ночей в среднем бронируют отели разных типов?

In [9]:
bookings.groupby('hotel').agg({'stays_total_nights': 'mean'}).reset_index()

,hotel,stays_total_nights
0,City Hotel,2.978142
1,Resort Hotel,4.318547


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?

In [14]:
bookings.query("assigned_room_type != reserved_room_type").shape

(14917, 21)

Проанализируйте даты запланированного прибытия.

– На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?

– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов.

In [41]:

bookings.groupby(['arrival_date_year','arrival_date_month']).count().reset_index().sort_values(by=['arrival_date_year', 'is_canceled'])

,arrival_date_year,arrival_date_month,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
3,2015,November,2340,2340,2340,2340,2340,2340,2340,2340,...,2340,2340,2340,2340,2320,2340,2340,2340,2340,2340
2,2015,July,2776,2776,2776,2776,2776,2776,2776,2776,...,2776,2776,2776,2776,2763,2776,2776,2776,2776,2776
1,2015,December,2920,2920,2920,2920,2920,2920,2920,2920,...,2920,2920,2920,2920,2900,2920,2920,2920,2920,2920
0,2015,August,3889,3889,3889,3889,3889,3889,3889,3889,...,3889,3885,3889,3889,3877,3889,3889,3889,3889,3889
4,2015,October,4957,4957,4957,4957,4957,4957,4957,4957,...,4957,4957,4957,4957,4919,4957,4957,4957,4957,4957
5,2015,September,5114,5114,5114,5114,5114,5114,5114,5114,...,5114,5114,5114,5114,5084,5114,5114,5114,5114,5114
10,2016,January,2248,2248,2248,2248,2248,2248,2248,2248,...,2248,2248,2248,2248,2215,2248,2248,2248,2248,2248
8,2016,December,3860,3860,3860,3860,3860,3860,3860,3860,...,3860,3860,3860,3860,3828,3860,3860,3860,3860,3860
9,2016,February,3891,3891,3891,3891,3891,3891,3891,3891,...,3891,3891,3891,3891,3857,3891,3891,3891,3891,3891
15,2016,November,4454,4454,4454,4454,4454,4454,4454,4454,...,4454,4454,4454,4454,4432,4454,4454,4454,4454,4454


**По данным - самый успешный месяц тот,в котором больше брони\
Этот месяц в 2016 году - Октябрь, в 2017 - Май**

In [32]:
# Добавляем в группировку еще один элемент
bookings.groupby(['arrival_date_year','arrival_date_month','hotel']).agg({'is_canceled': 'sum'}).reset_index().sort_values(by=['arrival_date_year','hotel', 'is_canceled'])

,arrival_date_year,arrival_date_month,hotel,is_canceled
6,2015,November,City Hotel,301
2,2015,December,City Hotel,668
4,2015,July,City Hotel,939
0,2015,August,City Hotel,1232
8,2015,October,City Hotel,1321
10,2015,September,City Hotel,1543
7,2015,November,Resort Hotel,185
3,2015,December,Resort Hotel,305
5,2015,July,Resort Hotel,320
1,2015,August,Resort Hotel,366


**Если я все правильно понял, то отмена по периодам - это самый неудачный месяц в году\
Если так то:\
2015 - Сентябрь, 2016 - Октябрь, 2017 - Май**